# **Proyecto de grado**










## *Aplicación móvil en Android con OCR para la lectura del consumo en medidores eléctricos de clientes residenciales en Santiago de Cali.*

### **Integrantes**


*   Daniel Alejandro Cerquera Castro
*   Brayan Starlin Garcés Portillo
*   Juan Camilo Vélez Olaya

### **Tutor**

*   Carlos Alberto Arce Lopera

En este Colab o Notebook, se explican cada uno de los pasos para la creación de un modelo de machine learning que detecte y reconozca los digitos que muestran el consumo de un medidor eléctrico.

# **Requerimientos**



1.   Un dataset con el que se entrenará el modelo. En este proyecto se utiliza un dataset de 2300 imágenes.
2.   El etiquetado del dataset.
3.   Configuración del pipeline para entrenar el modelo.

# **Procedimiento**

## **Setup**

In [ ]:
# Verificamos versiones de Python y TensorFlow (Python 3.7.10 y TensorFlow 2.4.1)
!python3 --version
import tensorflow as tf
print(tf.version.VERSION)

In [ ]:
# Se verifica la TPU.

print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Primero montamos Google Drive para crear nuestras carpetas y archivos necesarios para el pipeline.

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Clonamos el repo de TensorFlow Models para obtener el Object Detection API

%cd '/content/gdrive/My Drive/TensorFlow'

!git clone https://github.com/tensorflow/models.git

# Se usa una version anterior por compatibilidad.
%cd '/content/gdrive/MyDrive/TensorFlow/models'
!git checkout -f c3d53565f3b5964e7ab46cb9a23689ceacc8823a

In [ ]:
# Se instalan algunas librerias necesarias para el preprocesamiento del dataset.

!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

In [ ]:
# Se compilan las librerias de Protobuf

%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# Se configura la variable de entorno.

import os
import sys
os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/TensorFlow/models"
sys.path.append("/content/gdrive/My Drive/TensorFlow/models/research")

In [ ]:
# Se hace el build y la instalacion del object detection api.

!python setup.py build
!python setup.py install

In [ ]:
# Se comprueba que todo haya sido instalado correctamente

%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

## **Pre procesamiento**

In [ ]:
# Se generan los .record para realizar el entrenamiento y validacion. (El dataset ya se debe encontrar etiquetado)

%cd '/content/gdrive/My Drive/TensorFlow/scripts/preprocessing'

# Se genera el train.record para entrenar y el test.record para validar.
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/images/train' -l '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/train.record'
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/images/test' -l '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/test.record'


## **Training**

In [ ]:
# Iniciamos TensorBoard para visualizar algunas metricas.
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

# Para evauluar el modelo.
#!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config --checkpoint_dir=models/my_ssd_resnet50_v1_fpn
#%load_ext tensorboard
#%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn/eval

# Para hacer tracking mediante el entrenamiento.
%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn

In [ ]:
# Para checkear tiempo restante en el colab.

import time,psutil
uptime=time.time()-psutil.boot_time()
remaintime=(12*60*60)-uptime
print(remaintime/(60*60))

In [ ]:
# Entrenar el modelo
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config --use_tpu=true

## **Exportar el modelo**

In [ ]:
# Exportar el modelo entrenado.

!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn/ --output_directory ./exported-models/my_model

## **Testing**

In [ ]:
# Testear el modelo

import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/my_model/saved_model"

print('Loading model...', end='')

detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')

In [ ]:
# Se obtienen los labels con los que se etiquetó el modelo.

category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt",use_display_name=True)

In [ ]:
# Cargamos algunas imagenes para comprobar su funcionamiento.
img=['/content/img1.jpg','/content/img2.jpg']
print(img)

In [ ]:
# Corremos la inferencia del modelo

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

for image_path in img:

    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor=tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections=detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}
    detections['num_detections']=num_detections

    # detection_classes should be ints.
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    image_np_with_detections=image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=10,     #max number of bounding boxes in the image
          min_score_thresh=.5,      #min prediction threshold
          agnostic_mode=False)
    %matplotlib inline
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.show()